# Day 09. Exercise 03
# Ensembles

## 0. Imports

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import (
  train_test_split,
  GridSearchCV,
  StratifiedKFold,
  KFold,
  ParameterGrid,
)

from sklearn.linear_model import (
  LogisticRegression
)

from sklearn.tree import (
  DecisionTreeClassifier,
)

from sklearn.svm import (
  SVC
)

from sklearn.metrics import (
  accuracy_score,
  precision_score,
  recall_score,
  roc_auc_score,
  confusion_matrix,
)


from sklearn.ensemble import (
  RandomForestClassifier,
  VotingClassifier,
  BaggingClassifier,
  StackingClassifier,
)

from typing import (
  List,
  Dict
)

from itertools import product

import joblib

from tqdm.notebook import tqdm

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test` and then get `X_train`, `y_train`, `X_valid`, `y_valid` from the previous `X_train`, `y_train`. Use the additional parameter `stratify`.

In [3]:
df = pd.read_csv('../../datasets/day-of-week-not-scaled.csv')
df['dayofweek'] = pd.read_csv('../../datasets/dayofweek.csv')['dayofweek']
df

,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1,dayofweek
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,9,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
1682,6,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
1683,7,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
1684,8,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
  df.drop('dayofweek', axis=1),
  df['dayofweek'],
  stratify=df['dayofweek'],
  test_size=0.2,
  random_state=21,
  shuffle=True,
)

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(
  X_train,
  y_train,
  stratify=y_train,
  test_size=0.2,
  random_state=21,
  shuffle=True,
)

## 2. Individual classifiers

1. Train SVM, decision tree and random forest again with the best parameters that you got from the 01 exercise with `random_state=21` for all of them.
2. Evaluate `accuracy`, `precision`, and `recall` for them on the validation set.
3. The result of each cell of the section should look like this:

```
accuracy is 0.87778
precision is 0.88162
recall is 0.87778
```

In [6]:
%store -r svc_params
%store -r tree_params
%store -r forest_params

params = [svc_params, tree_params, forest_params]
params

[{'C': 10,
  'class_weight': None,
  'gamma': 'auto',
  'kernel': 'rbf',
  'probability': True,
  'random_state': 21},
 {'class_weight': 'balanced',
  'criterion': 'gini',
  'max_depth': 21,
  'random_state': 21},
 {'class_weight': 'balanced',
  'criterion': 'entropy',
  'max_depth': 24,
  'n_estimators': 100,
  'random_state': 21}]

In [7]:
def calculate_metrics (y_true, y_pred): 
  return {
  'accuracy': accuracy_score(y_true, y_pred),
  'precision': precision_score(y_true, y_pred, average='weighted'),
  'recall': recall_score(y_true, y_pred, average='weighted'),
  }

def print_metrics(metrics):
  for k, v in metrics.items():
    print(f'{k} is {v:.5f}')

In [8]:
def choose_params(estimator, estimator_params, X_train, y_train, X_valid, y_valid):
  
  params = ParameterGrid(estimator_params)

  best_param = None
  best_accuracy = 0.
  for param in tqdm(params, desc='Поиск лучших параметров'):
    estimator.set_params(**param)
    estimator.fit(X_train, y_train)
    metric = calculate_metrics(y_valid, estimator.predict(X_valid))

    if metric['accuracy'] > best_accuracy:
      best_accuracy = metric['accuracy']
      best_param = param

  estimator.set_params(**best_param)
  estimator.fit(X_train, y_train)

  return estimator

In [9]:
svc = SVC(**params[0])
svc.fit(X_train, y_train)
predict = pd.DataFrame(svc.predict(X_valid), columns=['svc_predict'])
predict['y_valid'] = y_valid

print_metrics(calculate_metrics(y_valid, predict['svc_predict']))


accuracy is 0.87778
precision is 0.88162
recall is 0.87778


In [10]:
tree = DecisionTreeClassifier(**params[1])
tree.fit(X_train, y_train)
predict = pd.DataFrame(tree.predict(X_valid), columns=['tree_predict'])
predict['y_valid'] = y_valid

print_metrics(calculate_metrics(y_valid, predict['tree_predict']))

accuracy is 0.86667
precision is 0.87170
recall is 0.86667


In [11]:
forest = RandomForestClassifier(**params[2])
forest.fit(X_train, y_train)
predict = pd.DataFrame(forest.predict(X_valid), columns=['forest_predict'])
predict['y_valid'] = y_valid

print_metrics(calculate_metrics(y_valid, predict['forest_predict']))

accuracy is 0.89630
precision is 0.89698
recall is 0.89630


## 3. Voting classifiers

1. Using `VotingClassifier` and the three models that you have just trained, calculate the `accuracy`, `precision`, and `recall` on the validation set.
2. Play with the other parameteres.
3. Calculate the `accuracy`, `precision` and `recall` on the test set for the model with the best weights in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).

In [12]:
voting = VotingClassifier(
  estimators=[('svc', svc), ('tree', tree), ('forest', forest)]
)
voting.fit(X_train, y_train);

In [13]:
print_metrics(calculate_metrics(y_valid, voting.predict(X_valid)))

accuracy is 0.90000
precision is 0.89993
recall is 0.90000


In [14]:

voting_params = {
  'voting': ['hard', 'soft'],
  'weights' : [list(w) for w in product(range(1,5), repeat=3)]
}
best_voting = choose_params(voting,voting_params, X_train, y_train, X_valid, y_valid)

Поиск лучших параметров:   0%|          | 0/128 [00:00<?, ?it/s]

In [15]:
best_voting.get_params(deep=False)

{'estimators': [('svc',
   SVC(C=10, gamma='auto', probability=True, random_state=21)),
  ('tree',
   DecisionTreeClassifier(class_weight='balanced', max_depth=21, random_state=21)),
  ('forest',
   RandomForestClassifier(class_weight='balanced', criterion='entropy',
                          max_depth=24, random_state=21))],
 'flatten_transform': True,
 'n_jobs': None,
 'verbose': False,
 'voting': 'soft',
 'weights': [4, 1, 4]}

In [16]:
print_metrics(calculate_metrics(y_test, best_voting.predict(X_test)))

accuracy is 0.90533
precision is 0.90881
recall is 0.90533


## 4. Bagging classifiers

1. Using `BaggingClassifier` and `SVM` with the best parameters create an ensemble, try different values of the `n_estimators`, use `random_state=21`.
2. Play with the other parameters.
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision)

In [17]:
bagging = BaggingClassifier(
  svc,
  random_state=21,
  verbose=1
)

In [18]:
bagging_params = {
  'n_estimators':[5, 10, 25, 50, 100, 200],
}

best_bagging = choose_params(bagging, bagging_params, X_train, y_train, X_test, y_test)


Поиск лучших параметров:   0%|          | 0/6 [00:00<?, ?it/s]

In [19]:
best_bagging.get_params(deep=False)

{'base_estimator': SVC(C=10, gamma='auto', probability=True, random_state=21),
 'bootstrap': True,
 'bootstrap_features': False,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 50,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 21,
 'verbose': 1,
 'warm_start': False}

In [20]:
print_metrics(calculate_metrics(y_test, best_bagging.predict(X_test)))

accuracy is 0.88462
precision is 0.88941
recall is 0.88462


## 5. Stacking classifiers

1. To achieve reproducibility in this case you will have to create an object of cross-validation generator: `StratifiedKFold(n_splits=n, shuffle=True, random_state=21)`, where `n` you will try to optimize (the details are below).
2. Using `StackingClassifier` and the three models that you have recently trained, calculate the `accuracy`, `precision` and `recall` on the validation set, try different values of `n_splits` `[2, 3, 4, 5, 6, 7]` in the cross-validation generator and parameter `passthrough` in the classifier itself,
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision). Use `final_estimator=LogisticRegression(solver='liblinear')`.

In [21]:
stacking = StackingClassifier(
  estimators=[('svc', svc), ('tree', tree), ('forest', forest)],
  final_estimator=LogisticRegression(solver='liblinear')
)

stacking_params = {
  'passthrough': [True, False],
  'cv' : [StratifiedKFold(n_splits=n, shuffle=True, random_state=21) for n in range(2,8)]
}

best_stacking = choose_params(stacking, stacking_params, X_train, y_train, X_valid, y_valid)


Поиск лучших параметров:   0%|          | 0/12 [00:00<?, ?it/s]

In [22]:
print_metrics(calculate_metrics(y_test, best_stacking.predict(X_test)))

accuracy is 0.90533
precision is 0.90844
recall is 0.90533


In [23]:
best_stacking.get_params(deep=False)

{'cv': StratifiedKFold(n_splits=4, random_state=21, shuffle=True),
 'estimators': [('svc',
   SVC(C=10, gamma='auto', probability=True, random_state=21)),
  ('tree',
   DecisionTreeClassifier(class_weight='balanced', max_depth=21, random_state=21)),
  ('forest',
   RandomForestClassifier(class_weight='balanced', criterion='entropy',
                          max_depth=24, random_state=21))],
 'final_estimator': LogisticRegression(solver='liblinear'),
 'n_jobs': None,
 'passthrough': True,
 'stack_method': 'auto',
 'verbose': 0}

## 6. Predictions

1. Choose the best model in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).
2. Analyze: for which weekday your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which labname and for which users.
3. Save the model.

In [26]:
best_results = pd.DataFrame(
  {
    'dayofweek' : y_test,
    'best_predict' : best_voting.predict(X_test),
    'errors' : y_test != best_voting.predict(X_test)
  }
)

sums = best_results.groupby('dayofweek').sum()

sums['errors percentage'] = sums['errors'] / sums['best_predict'] * 100
sums['errors percentage'].sort_values(ascending=False)


dayofweek
0    21.621622
1     6.451613
2     3.125000
5     2.583026
4     2.325581
6     1.442308
3     1.234568
Name: errors percentage, dtype: float64

In [25]:
joblib.dump(best_voting, 'model.joblib');